In [1]:
data = [
    "1,0,1~1,2,1",
    "0,0,2~2,0,2",
    "0,2,3~2,2,3",
    "0,0,4~0,2,4",
    "2,0,5~2,2,5",
    "0,1,6~2,1,6",
    "1,1,8~1,1,9",
]

In [37]:
with open("input") as f:
    data = [line.strip() for line in f]

In [38]:
class Brick:
    def __init__(self, x: range, y: range, z: range, n: int):
        self.x = x
        self.y = y
        self.z = z
        self.n = n

    def __repr__(self):
        c = "ABCDEFG"[self.n]
        return f"<Brick {c} x={self.x} y={self.y} z={self.z}>"

    @classmethod
    def from_coords(cls, c1, c2, n):
        (x1, y1, z1) = c1
        (x2, y2, z2) = c2
        x = range(x1, x2 + 1)
        y = range(y1, y2 + 1)
        z = range(z1, z2 + 1)
        return cls(x, y, z, n)

    @property
    def is_grounded(self):
        return 1 in self.z
    
    @property
    def new_brick_lower(self):
        z1 = min(self.z)
        if z1 == 1:
            return self
        z2 = max(self.z)
        z = range(z1 - 1, z2)
        return Brick(self.x, self.y, z, self.n)

In [39]:
def make_brick(coords, n):
    c1, c2 = coords
    return Brick.from_coords(c1, c2, n)

def make_bricks():
    return [
        make_brick((tuple(int(i) for i in c.split(",")) for c in line.split("~")), n)
        for n, line in enumerate(data)
    ]

In [41]:
def make_grid(bricks, z):
    grid = set()
    try:
        min_x = min(min(b.x) for b in bricks)
        max_x = max(max(b.x) for b in bricks)
        min_y = min(min(b.y) for b in bricks)
        max_y = max(max(b.y) for b in bricks)
    except ValueError:
        return grid
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            for b in bricks:
                if x in b.x and y in b.y and z in b.z:
                    assert (x, y) not in grid
                    grid.add((x, y))
    return grid

def collides(brick, other_bricks):
    for z in brick.z:
        grid = make_grid(other_bricks, z)
        for x in brick.x:
            for y in brick.y:
                if (x, y) in grid:
                    return True
    return False

def get_other_bricks(bricks, brick):
    return [b for b in bricks if b.n != brick.n]

def iter_gravity(bricks):
    for brick in bricks:
        if brick.is_grounded:
            yield (brick, True)
            continue
        new_brick = brick.new_brick_lower
        other_bricks = get_other_bricks(bricks, brick)
        if collides(new_brick, other_bricks):
            yield (brick, True)
        else:
            yield (new_brick, False)

def do_gravity(bricks):
    while True:
        update = list(iter_gravity(bricks))
        bricks = [b for b, status in update]
        if all([status for b, status in update]):
            return bricks
        
def this_brick_can_disintegrate(brick, bricks):
    other_bricks = get_other_bricks(bricks, brick)
    return all([status for b, status in iter_gravity(other_bricks)])

In [42]:
bricks = make_bricks()
bricks = list(do_gravity(bricks))
sum(this_brick_can_disintegrate(brick, bricks) for brick in bricks)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ben/.virtualenvs/aoc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_164272/4020863724.py", line 2, in <module>
    bricks = list(do_gravity(bricks))
                  ^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_164272/231494893.py", line 44, in do_gravity
    update = list(iter_gravity(bricks))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_164272/231494893.py", line 37, in iter_gravity
    if collides(new_brick, other_bricks):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_164272/231494893.py", line 20, in collides
    grid = make_grid(other_bricks, z)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_164272/231494893.py", line None, in make_grid
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home

In [43]:
len(bricks)

1203

In [6]:
def draw_xy(z):
    print(f"{z=}")
    grid = make_grid(z, bricks)
    if not grid:
        return
    min_x = min(x for x, y in grid)
    max_x = max(x for x, y in grid)
    min_y = min(y for x, y in grid)
    max_y = max(y for x, y in grid)
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            print("X" if (x, y) in grid else " ", end="")
        print()
    print()

In [7]:
# for z in range(10):
#     draw_xy(z)

In [8]:
# def draw(z):
#     print(f"{z=}")
#     for y in range(3):
#         for x in range(3):
#             c = " "
#             for b in bricks:
#                 if x in b.x and y in b.y and z in b.z:
#                     c = "X"
#             print(c, end="")
#         print()
#     print()